In [1]:
import tensorflow_datasets as tfds

ds_test = tfds.load('imdb_reviews', split='test[:100]', as_supervised=True)

textes = []
labels = []

for text, label in tfds.as_numpy(ds_test):
    textes.append(text.decode('utf-8'))
    labels.append(int(label))  

2025-04-30 13:44:18.858248: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746020659.129612      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746020659.205950      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.XT1TVQ_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.XT1TVQ_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


I0000 00:00:1746020723.133237      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1746020723.133885      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [2]:
print(labels)

[1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0]


In [3]:
import tensorflow as tf

model_path = "/kaggle/input/tuto_bert/keras/default/1/tuto_BERT"

model_bert = tf.saved_model.load(model_path)

In [4]:
from transformers import BertTokenizer
import tensorflow as tf
import numpy as np

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

inputs = tokenizer(
    textes,
    return_tensors="tf",
    padding=True,
    truncation=True
)

output = model_bert(inputs)
logits = output["logits"]

probas = tf.nn.softmax(logits, axis=-1)
preds = tf.argmax(probas, axis=1)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [5]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(labels, preds)

print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.91


In [6]:
!pip install mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.5/301.5 kB 11.3 MB/s eta 0:00:00


In [7]:
from mistralai import Mistral
import time
api_key = "use your own"
model_mistral = "mistral-large-latest"

client = Mistral(api_key=api_key)

In [8]:
pred_mistral=[]
explications=[]
for opinion in textes:
    chat_response = client.chat.complete(
        model=model_mistral,
        messages=[
            {
                "role": "user",
                "content": f"Evaluate this movie review : '{opinion}'. Respond with 0 if the review is negative, 1 if it is positive. Respond only with 0 or 1, nothing else. Then provide a short explanation as to why you choose negative or positive. ",
            },
        ]
    )
    print(chat_response.choices[0].message.content)
    pred_mistral.append(int(chat_response.choices[0].message.content[0]))
    explications.append(chat_response.choices[0].message.content)
    time.sleep(5)

1

The review is positive because it uses highly praising language such as "absolutely amazing," "flawless film-making," and "considerable talent." The reviewer also compares the film favorably to other successful movies and mentions having laughed hard, indicating a positive reception.
1

Explanation: The review is positive because it uses praising language such as "excellent," "remarkable," "moving," and "urge anyone interested to seek it out." The reviewer also places 'Nazarin' high in the ranking of Bunuel's films, indicating a strong positive opinion.
0

Explanation: The review is overwhelmingly negative. The reviewer explicitly states that the movie is a "disaster," "laughless," and one of the "worst outings of 2008." They also compare it unfavorably to other poorly received films and express sympathy for the actors involved, indicating a strong negative opinion.
0

Explanation: The review is negative. The reviewer criticizes the movie for being overly sentimental, unrealistic, a

In [9]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(labels, pred_mistral)

print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.90


In [10]:
print(explications)

['1\n\nThe review is positive because it uses highly praising language such as "absolutely amazing," "flawless film-making," and "considerable talent." The reviewer also compares the film favorably to other successful movies and mentions having laughed hard, indicating a positive reception.', '1\n\nExplanation: The review is positive because it uses praising language such as "excellent," "remarkable," "moving," and "urge anyone interested to seek it out." The reviewer also places \'Nazarin\' high in the ranking of Bunuel\'s films, indicating a strong positive opinion.', '0\n\nExplanation: The review is overwhelmingly negative. The reviewer explicitly states that the movie is a "disaster," "laughless," and one of the "worst outings of 2008." They also compare it unfavorably to other poorly received films and express sympathy for the actors involved, indicating a strong negative opinion.', "0\n\nExplanation: The review is negative. The reviewer criticizes the movie for being overly senti

In [11]:
chat_response = client.chat.complete(
        model=model_mistral,
        messages=[
            {
                "role": "user",
                "content": f"Here is a list of explanations as to why 100 movie reviews were classified as they were : {explications}. Make a list of 10 bullet points of the main reasons why some reviews were classified as negative, and do the same for the positive ones but do not provide an explanation, just a reason for each bullet point",
            },
        ]
    )
print(chat_response.choices[0].message.content)

### Negative Reviews
- The reviewer explicitly states that they did not like the movie.
- The reviewer criticizes various aspects of the film, such as the characters, humor, dialogues, and the lack of expected elements.
- The reviewer describes the film as a "disaster," "laughless," and one of the "worst outings."
- The reviewer criticizes the movie for being overly sentimental, unrealistic, and predictable.
- The reviewer explicitly states that the writing, direction, acting, and editing are "abysmal at best."
- The reviewer describes the film as a "mess" with a "ludicrous" plot, "unfunny" comedy, and uninteresting lead characters.
- The reviewer uses derogatory language to describe the film, such as "sordid faux children's film," "bitter and cold-hearted father," and "regrettably over-utilized."
- The reviewer hated the film, found it stupid, unfunny, and a waste of time.
- The reviewer describes the movie as "dull," "poor," "stereotypical," and a "disaster."
- The reviewer states th